No need to run ,it may exceed the diskspace. Need restart

In [ ]:
import os
# Set the environment variable
os.environ['HF_HOME'] = "/kaggle/working/cache/"
print('done')

In [ ]:
!pip -q install huggingface_hub diffusers einops

In [ ]:
!ls

In [ ]:
# Clone the InstantStyle repository
!git clone https://github.com/InstantStyle/InstantStyle.git

In [1]:
# Change directory to InstantStyle
%cd InstantStyle

/kaggle/working/InstantStyle


**Local Lora:**

In [ ]:
from huggingface_hub import hf_hub_download
import os
style_B_LoRA_path=hf_hub_download(repo_id="ElaineYin/fangao1", filename="f1_1000.safetensors", local_dir="lora100")
print(style_B_LoRA_path)

In [ ]:
from huggingface_hub import hf_hub_download
import os
ip_ckpt=hf_hub_download(repo_id="h94/IP-Adapter", filename="sdxl_models/ip-adapter_sdxl.bin", local_dir="ipadapter")
print(ip_ckpt)
hf_hub_download(repo_id="h94/IP-Adapter", filename="sdxl_models/image_encoder/config.json", local_dir="ipadapter")
hf_hub_download(repo_id="h94/IP-Adapter", filename="sdxl_models/image_encoder/pytorch_model.bin", local_dir="ipadapter")
image_encoder_file_path=hf_hub_download(repo_id="h94/IP-Adapter", filename="sdxl_models/image_encoder/model.safetensors", local_dir="ipadapter")
# Get the parent folder of the file
image_encoder_path = os.path.dirname(image_encoder_file_path)

# Print the parent folder
print(image_encoder_path)
# hf_hub_download(repo_id="h94/IP-Adapter", filename="sdxl_models/image_encoder/config.json")

In [2]:
ip_ckpt='ipadapter/sdxl_models/ip-adapter_sdxl.bin'
print(ip_ckpt)
image_encoder_path='ipadapter/sdxl_models/image_encoder'
print(image_encoder_path)

ipadapter/sdxl_models/ip-adapter_sdxl.bin
ipadapter/sdxl_models/image_encoder


In [8]:
import torch
from diffusers import StableDiffusionXLPipeline
from PIL import Image

from ip_adapter import IPAdapterXL

BLOCKS_M = {
    # target_blocks=["block"] for original IP-Adapter
    # target_blocks=["up_blocks.0.attentions.1"] for style blocks only
    # target_blocks = ["up_blocks.0.attentions.1", "down_blocks.2.attentions.1"] # for style+layout blocks
    
    'ALL': ['block'],
    'Wm1':['down_blocks.1.attentions.0'],
    'W0':['down_blocks.1.attentions.1'],
    'W1':['down_blocks.2.attentions.0'],
    'W2':['down_blocks.2.attentions.1'],
    'W3':['mid_block.attentions.0'],
    'W4':['up_blocks.0.attentions.0'],
    'W5':['up_blocks.0.attentions.1'],
    'W6':['up_blocks.0.attentions.2'],
    'W7':['up_blocks.1.attentions.0'],
    'W8':['up_blocks.1.attentions.1'],
    'W9':['up_blocks.1.attentions.2'],
    
    'W20':['down_blocks.2.attentions.1.transformer_blocks.0'],
    'W21':['down_blocks.2.attentions.1.transformer_blocks.1'],
    'W22':['down_blocks.2.attentions.1.transformer_blocks.2'],
    'W23':['down_blocks.2.attentions.1.transformer_blocks.3'],
    'W24':['down_blocks.2.attentions.1.transformer_blocks.4'],
    'W25':['down_blocks.2.attentions.1.transformer_blocks.5'],
    'W26':['down_blocks.2.attentions.1.transformer_blocks.6'],
    'W27':['down_blocks.2.attentions.1.transformer_blocks.7'],
    'W28':['down_blocks.2.attentions.1.transformer_blocks.8'],
    'W29':['down_blocks.2.attentions.1.transformer_blocks.9'],
    'W50':['up_blocks.0.attentions.1.transformer_blocks.0'],
    'W51':['up_blocks.0.attentions.1.transformer_blocks.1'],
    'W52':['up_blocks.0.attentions.1.transformer_blocks.2'],
    'W53':['up_blocks.0.attentions.1.transformer_blocks.3'],
    'W54':['up_blocks.0.attentions.1.transformer_blocks.4'],
    'W55':['up_blocks.0.attentions.1.transformer_blocks.5'],
    'W56':['up_blocks.0.attentions.1.transformer_blocks.6'],
    'W57':['up_blocks.0.attentions.1.transformer_blocks.7'],
    'W58':['up_blocks.0.attentions.1.transformer_blocks.8'],
    'W59':['up_blocks.0.attentions.1.transformer_blocks.9'],
    'W60':['up_blocks.0.attentions.2.transformer_blocks.0'],
    'W61':['up_blocks.0.attentions.2.transformer_blocks.1'],
    'W62':['up_blocks.0.attentions.2.transformer_blocks.2'],
    'W63':['up_blocks.0.attentions.2.transformer_blocks.3'],
    'W64':['up_blocks.0.attentions.2.transformer_blocks.4'],
    'W65':['up_blocks.0.attentions.2.transformer_blocks.5'],
    'W66':['up_blocks.0.attentions.2.transformer_blocks.6'],
    'W67':['up_blocks.0.attentions.2.transformer_blocks.7'],
    'W68':['up_blocks.0.attentions.2.transformer_blocks.8'],
    'W69':['up_blocks.0.attentions.2.transformer_blocks.9'],
    
}
base_model_path = "stabilityai/stable-diffusion-xl-base-1.0"
# image_encoder_path = "sdxl_models/image_encoder"
# ip_ckpt = "sdxl_models/ip-adapter_sdxl.bin"
device = "cuda"

# Function to free up GPU memory
def freeCache(pipeline):
    try:
        del pipeline  # Delete the pipeline object if it exists
#         print("Pipeline deleted to free up GPU memory.")
    except NameError:
        print("Pipeline does not exist, no need to delete.")
    except Exception as e:
        print(f"An error occurred when trying to delete pipeline: {e}")
    
    torch.cuda.empty_cache()  # Clear the GPU memory
    
# Optionally clear cache for transformers
import transformers
transformers.utils.move_cache()
    
def genImg(layers,refImg,styleKey,objectNames,neg_conten_prompt=''):
    layerList=layers.split('_')
    
    blocks=[]
    for lay in layerList:
        blocks.extend(BLOCKS_M[lay])
        
    # load SDXL pipeline
    pipe = StableDiffusionXLPipeline.from_pretrained(
        base_model_path,
        torch_dtype=torch.float16,
        add_watermarker=False,
    )

    # reduce memory consumption
    pipe.enable_vae_tiling()
    
#     target_blocks=BLOCKS_M[block_key]
    # load ip-adapter
    ip_model = IPAdapterXL(pipe, image_encoder_path, 
                           ip_ckpt, device, target_blocks=blocks)

#     image = "./assets/0.jpg"
    image = Image.open(refImg)
    image.resize((512, 512))
    
#     neg_content_scaleli=[0.5,0.7]
    neg_content_scaleli=[0.3,0.5,0.7]
    negative_prompt= f'text, watermark, lowres, low quality, worst quality, deformed, glitch, low contrast, noisy, saturation, blurry'
    print('negative_prompt :',negative_prompt," neg_conten_prompt:",neg_conten_prompt)
    for neg_content_scale in neg_content_scaleli:
        for objectName in objectNames:
            # generate image variations with only image prompt
            images = ip_model.generate(pil_image=image,
                                        prompt=f'a {objectName}, masterpiece, best quality, high quality',
                                        negative_prompt= negative_prompt,
                                        scale=1.0,
                                        guidance_scale=5,
                                        num_samples=1,
                                        num_inference_steps=30, 
                                        seed=42,
                                        neg_content_prompt=f'{neg_conten_prompt}',
                                        neg_content_scale=neg_content_scale,
                                      )
            print(f' done layer: {layers}')
            images[0].save(f'{styleKey}__{neg_conten_prompt}_{str(neg_content_scale)}_ins_{objectName}_{layers}_1000.png')
            freeCache(pipe)


0it [00:00, ?it/s]

In [9]:
layerList=['W2_W5']
# layerList=['W2_W5','W5']
# layerList=['Wm1','W0','W7','W8','W9',
#           'Wm1_W0','Wm1_W1','Wm1_W2','Wm1_W3','Wm1_W4','Wm1_W5','Wm1_W6','Wm1_W7','Wm1_W8','Wm1_W9',
#           'W0_W1','W0_W2','W0_W3','W0_W4','W0_W5','W0_W6','W0_W7','W0_W8','W0_W9',
#           'W1_W7','W1_W8','W1_W9',
#           'W2_W7','W2_W8','W2_W9',
#           'W3_W7','W3_W8','W3_W9',
#           'W4_W7','W4_W8','W4_W9',
#           'W5_W7','W5_W8','W5_W9',
#           'W6_W7','W6_W8','W6_W9',
#           'W7_W8','W7_W9',
#           'W8_W9',
#           'W1','W2','W3','W4','W5','W6','W1_W2','W1_W3','W1_W4','W1_W5','W1_W6','W2_W3','W2_W4','W2_W5','W2_W6','W3_W4','W3_W5','W3_W6','W4_W5','W4_W6','W5_W6'
#          ]
# layerList=[
# 'W20','W20_W21','W20_W21_W22','W20_W21_W22_W23','W20_W21_W22_W23_W24','W20_W21_W22_W23_W24_W25','W20_W21_W22_W23_W24_W25_W26','W20_W21_W22_W23_W24_W25_W26_W27','W20_W21_W22_W23_W24_W25_W26_W27_W28','W20_W21_W22_W23_W24_W25_W26_W27_W28_W29',
# 'W21','W21_W22','W21_W22_W23','W21_W22_W23_W24','W21_W22_W23_W24_W25','W21_W22_W23_W24_W25_W26','W21_W22_W23_W24_W25_W26_W27','W21_W22_W23_W24_W25_W26_W27_W28','W21_W22_W23_W24_W25_W26_W27_W28_W29',
# 'W22','W22_W23','W22_W23_W24','W22_W23_W24_W25','W22_W23_W24_W25_W26','W22_W23_W24_W25_W26_W27','W22_W23_W24_W25_W26_W27_W28','W22_W23_W24_W25_W26_W27_W28_W29',
# 'W23','W23_W24','W23_W24_W25','W23_W24_W25_W26','W23_W24_W25_W26_W27','W23_W24_W25_W26_W27_W28','W23_W24_W25_W26_W27_W28_W29',
# 'W24','W24_W25','W24_W25_W26','W24_W25_W26_W27','W24_W25_W26_W27_W28','W24_W25_W26_W27_W28_W29',
# 'W25','W25_W26','W25_W26_W27','W25_W26_W27_W28','W25_W26_W27_W28_W29',
# 'W26','W26_W27','W26_W27_W28','W26_W27_W28_W29',
# 'W27','W27_W28','W27_W28_W29',
# 'W28','W28_W29'
# ]
# layerList=[
# 'W50','W50_W51','W50_W51_W52','W50_W51_W52_W53','W50_W51_W52_W53_W54','W50_W51_W52_W53_W54_W55','W50_W51_W52_W53_W54_W55_W56','W50_W51_W52_W53_W54_W55_W56_W57','W50_W51_W52_W53_W54_W55_W56_W57_W58','W50_W51_W52_W53_W54_W55_W56_W57_W58_W59',
# 'W51','W51_W52','W51_W52_W53','W51_W52_W53_W54','W51_W52_W53_W54_W55','W51_W52_W53_W54_W55_W56','W51_W52_W53_W54_W55_W56_W57','W51_W52_W53_W54_W55_W56_W57_W58','W51_W52_W53_W54_W55_W56_W57_W58_W59',
# 'W52','W52_W53','W52_W53_W54','W52_W53_W54_W55','W52_W53_W54_W55_W56','W52_W53_W54_W55_W56_W57','W52_W53_W54_W55_W56_W57_W58','W52_W53_W54_W55_W56_W57_W58_W59',
# 'W53','W53_W54','W53_W54_W55','W53_W54_W55_W56','W53_W54_W55_W56_W57','W53_W54_W55_W56_W57_W58','W53_W54_W55_W56_W57_W58_W59',
# 'W54','W54_W55','W54_W55_W56','W54_W55_W56_W57','W54_W55_W56_W57_W58','W54_W55_W56_W57_W58_W59',
# 'W55','W55_W56','W55_W56_W57','W55_W56_W57_W58','W55_W56_W57_W58_W59',
# 'W56','W56_W57','W56_W57_W58','W56_W57_W58_W59',
# 'W57','W57_W58','W57_W58_W59',
# 'W58','W58_W59'
# ]
# layerList=[
# 'W50','W50_W51','W50_W51_W52','W50_W51_W52_W53','W50_W51_W52_W53_W54','W50_W51_W52_W53_W54_W55','W50_W51_W52_W53_W54_W55_W56','W50_W51_W52_W53_W54_W55_W56_W57','W50_W51_W52_W53_W54_W55_W56_W57_W58','W50_W51_W52_W53_W54_W55_W56_W57_W58_W59','W50_W51_W52_W53_W54_W55_W56_W57_W58_W59_W60','W50_W51_W52_W53_W54_W55_W56_W57_W58_W59_W60_W61','W50_W51_W52_W53_W54_W55_W56_W57_W58_W59_W60_W61_W62','W50_W51_W52_W53_W54_W55_W56_W57_W58_W59_W60_W61_W62_W63','W50_W51_W52_W53_W54_W55_W56_W57_W58_W59_W60_W61_W62_W63_W64',
# 'W51','W51_W52','W51_W52_W53','W51_W52_W53_W54','W51_W52_W53_W54_W55','W51_W52_W53_W54_W55_W56','W51_W52_W53_W54_W55_W56_W57','W51_W52_W53_W54_W55_W56_W57_W58','W51_W52_W53_W54_W55_W56_W57_W58_W59','W51_W52_W53_W54_W55_W56_W57_W58_W59_W60','W51_W52_W53_W54_W55_W56_W57_W58_W59_W60_W61','W51_W52_W53_W54_W55_W56_W57_W58_W59_W60_W61_W62','W51_W52_W53_W54_W55_W56_W57_W58_W59_W60_W61_W62_W63','W51_W52_W53_W54_W55_W56_W57_W58_W59_W60_W61_W62_W63_W64',
# 'W52','W52_W53','W52_W53_W54','W52_W53_W54_W55','W52_W53_W54_W55_W56','W52_W53_W54_W55_W56_W57','W52_W53_W54_W55_W56_W57_W58','W52_W53_W54_W55_W56_W57_W58_W59','W52_W53_W54_W55_W56_W57_W58_W59_W60','W52_W53_W54_W55_W56_W57_W58_W59_W60_W61','W52_W53_W54_W55_W56_W57_W58_W59_W60_W61_W62','W52_W53_W54_W55_W56_W57_W58_W59_W60_W61_W62_W63','W52_W53_W54_W55_W56_W57_W58_W59_W60_W61_W62_W63_W64',
# 'W53','W53_W54','W53_W54_W55','W53_W54_W55_W56','W53_W54_W55_W56_W57','W53_W54_W55_W56_W57_W58','W53_W54_W55_W56_W57_W58_W59','W53_W54_W55_W56_W57_W58_W59_W60','W53_W54_W55_W56_W57_W58_W59_W60_W61','W53_W54_W55_W56_W57_W58_W59_W60_W61_W62','W53_W54_W55_W56_W57_W58_W59_W60_W61_W62_W63','W53_W54_W55_W56_W57_W58_W59_W60_W61_W62_W63_W64',
# 'W54','W54_W55','W54_W55_W56','W54_W55_W56_W57','W54_W55_W56_W57_W58','W54_W55_W56_W57_W58_W59','W54_W55_W56_W57_W58_W59_W60','W54_W55_W56_W57_W58_W59_W60_W61','W54_W55_W56_W57_W58_W59_W60_W61_W62','W54_W55_W56_W57_W58_W59_W60_W61_W62_W63','W54_W55_W56_W57_W58_W59_W60_W61_W62_W63_W64',
# 'W55','W55_W56','W55_W56_W57','W55_W56_W57_W58','W55_W56_W57_W58_W59','W55_W56_W57_W58_W59_W60','W55_W56_W57_W58_W59_W60_W61','W55_W56_W57_W58_W59_W60_W61_W62','W55_W56_W57_W58_W59_W60_W61_W62_W63','W55_W56_W57_W58_W59_W60_W61_W62_W63_W64',
# 'W56','W56_W57','W56_W57_W58','W56_W57_W58_W59','W56_W57_W58_W59_W60','W56_W57_W58_W59_W60_W61','W56_W57_W58_W59_W60_W61_W62','W56_W57_W58_W59_W60_W61_W62_W63','W56_W57_W58_W59_W60_W61_W62_W63_W64',
# 'W57','W57_W58','W57_W58_W59','W57_W58_W59_W60','W57_W58_W59_W60_W61','W57_W58_W59_W60_W61_W62','W57_W58_W59_W60_W61_W62_W63','W57_W58_W59_W60_W61_W62_W63_W64',
# 'W58','W58_W59','W58_W59_W60','W58_W59_W60_W61','W58_W59_W60_W61_W62','W58_W59_W60_W61_W62_W63','W58_W59_W60_W61_W62_W63_W64'
# ]
# layerList=[
# 'W50_W51_W52_W53_W54_W55_W56_W57_W58_W59_W60','W50_W51_W52_W53_W54_W55_W56_W57_W58_W59_W60_W61','W50_W51_W52_W53_W54_W55_W56_W57_W58_W59_W60_W61_W62','W50_W51_W52_W53_W54_W55_W56_W57_W58_W59_W60_W61_W62_W63','W50_W51_W52_W53_W54_W55_W56_W57_W58_W59_W60_W61_W62_W63_W64',
# 'W51_W52_W53_W54_W55_W56_W57_W58_W59_W60','W51_W52_W53_W54_W55_W56_W57_W58_W59_W60_W61','W51_W52_W53_W54_W55_W56_W57_W58_W59_W60_W61_W62','W51_W52_W53_W54_W55_W56_W57_W58_W59_W60_W61_W62_W63','W51_W52_W53_W54_W55_W56_W57_W58_W59_W60_W61_W62_W63_W64',
# 'W52_W53_W54_W55_W56_W57_W58_W59_W60','W52_W53_W54_W55_W56_W57_W58_W59_W60_W61','W52_W53_W54_W55_W56_W57_W58_W59_W60_W61_W62','W52_W53_W54_W55_W56_W57_W58_W59_W60_W61_W62_W63','W52_W53_W54_W55_W56_W57_W58_W59_W60_W61_W62_W63_W64',
# 'W53_W54_W55_W56_W57_W58_W59_W60','W53_W54_W55_W56_W57_W58_W59_W60_W61','W53_W54_W55_W56_W57_W58_W59_W60_W61_W62','W53_W54_W55_W56_W57_W58_W59_W60_W61_W62_W63','W53_W54_W55_W56_W57_W58_W59_W60_W61_W62_W63_W64',
# 'W54_W55_W56_W57_W58_W59_W60','W54_W55_W56_W57_W58_W59_W60_W61','W54_W55_W56_W57_W58_W59_W60_W61_W62','W54_W55_W56_W57_W58_W59_W60_W61_W62_W63','W54_W55_W56_W57_W58_W59_W60_W61_W62_W63_W64',
# 'W55_W56_W57_W58_W59_W60','W55_W56_W57_W58_W59_W60_W61','W55_W56_W57_W58_W59_W60_W61_W62','W55_W56_W57_W58_W59_W60_W61_W62_W63','W55_W56_W57_W58_W59_W60_W61_W62_W63_W64',
# 'W56_W57_W58_W59_W60','W56_W57_W58_W59_W60_W61','W56_W57_W58_W59_W60_W61_W62','W56_W57_W58_W59_W60_W61_W62_W63','W56_W57_W58_W59_W60_W61_W62_W63_W64',
# 'W57_W58_W59_W60','W57_W58_W59_W60_W61','W57_W58_W59_W60_W61_W62','W57_W58_W59_W60_W61_W62_W63','W57_W58_W59_W60_W61_W62_W63_W64',
# 'W58_W59_W60','W58_W59_W60_W61','W58_W59_W60_W61_W62','W58_W59_W60_W61_W62_W63','W58_W59_W60_W61_W62_W63_W64'
# ]
# layerList=['W58_W59_W60_W61_W62','W58_W59_W60_W61_W62_W63','W58_W59_W60_W61_W62_W63_W64']
#TMP
# layerList=['W22_W23_W24_W25_W26_W27_W28_W29']
# layerList=['W1_W2','W1_W3','W1_W4','W1_W5','W1_W6','W2_W6','W5_W6']
# refImg='/kaggle/input/img-f2/f2.PNG'
# refImg='/kaggle/input/img-grape/grape.png'
# styleKey='grape'
# neg_conten_prompt='grape'
# refImg='/kaggle/input/style-view/Weixin Image_20240511110827.png'
# neg_conten_prompt='people'
# styleKey='aut'
# refImg='/kaggle/input/img-bajiao/bajiao.png'
# neg_conten_prompt='girl'
# styleKey='bajiao'
refImg='/kaggle/input/img-colorbuilding/color_building.png'
styleKey='colorbuilding'
neg_conten_prompt='tower'
objectNames=['apple','dog','girl','fish','cat']
for layers in layerList:
    genImg(layers,refImg,styleKey,objectNames,neg_conten_prompt)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

negative_prompt : text, watermark, lowres, low quality, worst quality, deformed, glitch, low contrast, noisy, saturation, blurry  neg_conten_prompt: tower


  0%|          | 0/30 [00:00<?, ?it/s]

 done layer: W2_W5


  0%|          | 0/30 [00:00<?, ?it/s]

...

 done layer: W2_W5


  0%|          | 0/30 [00:00<?, ?it/s]

 done layer: W2_W5


  0%|          | 0/30 [00:00<?, ?it/s]

 done layer: W2_W5


# remove all

In [ ]:
!rm -rf /kaggle/working/

# ZIP

In [ ]:
#only for rp
!apt-get update
!apt-get install zip -y

In [10]:
envOutPath='/kaggle/working/InstantStyle'
# envOutPath='/content'
# envOutPath='/workspace'
!zip -j instan_neg12.zip {envOutPath}/*.png
# !zip a_images.zip {envOutPath}/B-LoRA/*.png

  adding: colorbuilding__tower_0.3_ins_apple_W2_W5_1000.png (deflated 0%)
  adding: colorbuilding__tower_0.3_ins_cat_W2_W5_1000.png (deflated 0%)
  adding: colorbuilding__tower_0.3_ins_dog_W2_W5_1000.png (deflated 0%)
  adding: colorbuilding__tower_0.3_ins_fish_W2_W5_1000.png (deflated 0%)
  adding: colorbuilding__tower_0.3_ins_girl_W2_W5_1000.png (deflated 0%)
  adding: colorbuilding__tower_0.5_ins_apple_W2_W5_1000.png (deflated 0%)
  adding: colorbuilding__tower_0.5_ins_cat_W2_W5_1000.png (deflated 0%)
  adding: colorbuilding__tower_0.5_ins_dog_W2_W5_1000.png (deflated 0%)
  adding: colorbuilding__tower_0.5_ins_fish_W2_W5_1000.png (deflated 0%)
  adding: colorbuilding__tower_0.5_ins_girl_W2_W5_1000.png (deflated 0%)
  adding: colorbuilding__tower_0.7_ins_apple_W2_W5_1000.png (deflated 0%)
  adding: colorbuilding__tower_0.7_ins_cat_W2_W5_1000.png (deflated 0%)
  adding: colorbuilding__tower_0.7_ins_dog_W2_W5_1000.png (deflated 0%)
  adding: colorbuilding__tower_0.7_ins_fish_W2_W5_1000

In [11]:
!rm {envOutPath}/*.png

In [ ]:
!rm {envOutPath}/*.zip

# only for test,no need to run:

In [ ]:
!pip -q install huggingface_hub diffusers einops

In [ ]:
from huggingface_hub import hf_hub_download
import os
ip_ckpt=hf_hub_download(repo_id="h94/IP-Adapter", filename="sdxl_models/ip-adapter_sdxl.bin", local_dir="ipadapter")
print(ip_ckpt)
hf_hub_download(repo_id="h94/IP-Adapter", filename="sdxl_models/image_encoder/config.json", local_dir="ipadapter")
hf_hub_download(repo_id="h94/IP-Adapter", filename="sdxl_models/image_encoder/pytorch_model.bin", local_dir="ipadapter")
image_encoder_file_path=hf_hub_download(repo_id="h94/IP-Adapter", filename="sdxl_models/image_encoder/model.safetensors", local_dir="ipadapter")
# Get the parent folder of the file
image_encoder_path = os.path.dirname(image_encoder_file_path)

# Print the parent folder
print(image_encoder_path)
# hf_hub_download(repo_id="h94/IP-Adapter", filename="sdxl_models/image_encoder/config.json")

In [ ]:

base_model_path = "stabilityai/stable-diffusion-xl-base-1.0"
pipe = StableDiffusionXLPipeline.from_pretrained(
        base_model_path,
        torch_dtype=torch.float16,
        add_watermarker=False,
    )

In [ ]:
!wget https://civitai.com/api/download/models/129573 -O 3DStyle.safetensors
!wget https://civitai.com/api/download/models/367359 -O blue_pencil-XL.safetensors
# !wget https://civitai.com/api/download/models/163062 -O hanfuSong.safetensors

In [ ]:
from safetensors.torch import load_file
import torch
from diffusers import StableDiffusionXLPipeline
# pipeline = StableDiffusionXLPipeline.from_pretrained(
#             "stabilityai/stable-diffusion-xl-base-1.0",
#             torch_dtype=torch.float16,
#         )
    
# Load the safetensors file
# state_dict = load_file("hanfuSong.safetensors")
# state_dict = load_file("3DStyle.safetensors")
state_dict = load_file("blue_pencil-XL.safetensors")

# Assuming you have a model class defined somewhere
# model = YourModelClass()

# Load the state_dict into the model
# style_B_LoRA_sd, _ =pipeline.lora_state_dict(state_dict)

# print(list(style_B_LoRA_sd.keys()))

# state_dict = pipeline.unet.state_dict()

# Print all the keys (names of layers/parameters)
for key in state_dict.keys():
    print(key)
# Set the model to evaluation mode
# model.eval()